# Upload Kaggle .json API

Upload file `kaggle.json` kalian

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"immanuelanthony","key":"1eb6ba8916d3efa2823e28dd8edf3009"}'}

# Loading Kaggle Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle # Make a directory for the .json file
! cp kaggle.json ~/.kaggle/ # Copy the .json file to the .kaggle directory
! chmod 600 ~/.kaggle/kaggle.json # GIve user permission to read/write file

In [ ]:
mkdir dataset # Make a directory for the dataset

In [ ]:
! kaggle datasets download -d yudhaislamisulistya/plants-type-datasets # Download the kaggle datasets

100% 2.69G/2.69G [00:32<00:00, 122MB/s] 
100% 2.69G/2.69G [00:32<00:00, 89.1MB/s]


In [ ]:
! unzip plants-type-datasets.zip -d dataset # Unzip the kaggle dataset zip to the 'dataset' directory

In [ ]:
path = '/content/dataset/split_ttv_dataset_type_of_plants/split_ttv_dataset_type_of_plants/' # Specify the main directory for the splitting of labels
train_path = 'Train_Set_Folder'
test_ds_path = 'Test_Set_Folder'
val_ds_path = 'Validation_Set_Folder'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Create the ImageDataGenerator class for augmenting the training datasets
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=True,
                brightness_range=(0.8, 1.2),
                fill_mode='nearest'
                )

train_ds = train_datagen.flow_from_directory(
          path + train_path,
          target_size=(255, 255),
          batch_size=32,
          class_mode='sparse'
          )

test_ds = tf.keras.utils.image_dataset_from_directory(
          path + test_ds_path,
          image_size=(255, 255),
          batch_size=32,
          label_mode = 'int'
          )

val_ds = tf.keras.utils.image_dataset_from_directory(
          path + val_ds_path,
          image_size=(255, 255),
          batch_size=32,
          label_mode = 'int'
          )

num_classes = len(train_ds.class_indices)
print(f"Jumlah tipe tanaman : {num_classes}")
print(f"label tipe tanaman : \n {train_ds.class_indices}")
# Nanti pake train_ds untuk fitting model

Found 24000 images belonging to 30 classes.
Found 3000 files belonging to 30 classes.
Found 3000 files belonging to 30 classes.
Jumlah tipe tanaman : 30
label tipe tanaman : 
 {'aloevera': 0, 'banana': 1, 'bilimbi': 2, 'cantaloupe': 3, 'cassava': 4, 'coconut': 5, 'corn': 6, 'cucumber': 7, 'curcuma': 8, 'eggplant': 9, 'galangal': 10, 'ginger': 11, 'guava': 12, 'kale': 13, 'longbeans': 14, 'mango': 15, 'melon': 16, 'orange': 17, 'paddy': 18, 'papaya': 19, 'peper chili': 20, 'pineapple': 21, 'pomelo': 22, 'shallot': 23, 'soybeans': 24, 'spinach': 25, 'sweet potatoes': 26, 'tobacco': 27, 'waterapple': 28, 'watermelon': 29}


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 253, 253, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

Epoch 1/10
750/750 [==============================] - 4191s 6s/step - loss: 2.8324 - accuracy: 0.1673 - val_loss: 213.6432 - val_accuracy: 0.1653
Epoch 2/10
750/750 [==============================] - 4197s 6s/step - loss: 2.3359 - accuracy: 0.2953 - val_loss: 264.6606 - val_accuracy: 0.1787
Epoch 3/10
750/750 [==============================] - 4139s 6s/step - loss: 2.1464 - accuracy: 0.3566 - val_loss: 386.0312 - val_accuracy: 0.1657
Epoch 4/10
750/750 [==============================] - 4182s 6s/step - loss: 2.0295 - accuracy: 0.3913 - val_loss: 304.3284 - val_accuracy: 0.1797
Epoch 5/10
750/750 [==============================] - 4164s 6s/step - loss: 1.9471 - accuracy: 0.4190 - val_loss: 312.1269 - val_accuracy: 0.2087
Epoch 6/10
750/750 [==============================] - 4143s 6s/step - loss: 1.8481 - accuracy: 0.4457 - val_loss: 350.1897 - val_accuracy: 0.1877
Epoch 7/10
750/750 [==============================] - 4128s 6s/step - loss: 1.7847 - accuracy: 0.4658 - val_loss: 310.0118 -

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

94/94 [==============================] - 135s 1s/step - loss: 267.0365 - accuracy: 0.2620
Test accuracy: 26.20%


In [ ]:
model.save('plant_classification_model.h5')

NameError: ignored